In [58]:
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

### DEEP LEARNING ###
from helper import *

## EXTRACTING X AND Y MATRICES. READ HELPER FOR FUNCTION INFO ##
dh = data_handle()

classes = (2,3,5) # Only 3 classes for now
img_size = 128

X, y = dh.featurize(resize=img_size, n=1000, include=classes)
num_classes = len(classes)

In [59]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
lb = preprocessing.LabelBinarizer()

range_ = np.array(list(set(y)))
out = lb.fit(range_)
y_binary = out.transform(y.astype(int))

# from sklearn.preprocessing import StandardScaler
# scalify = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
# X_train = scalify.fit_transform(X_train)
# X_test = scalify.fit_transform(X_test)

X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X, y_binary, test_size=0.20, random_state=42)
# X_train_b = scalify.fit_transform(X_train_b)
# X_test_b = scalify.fit_transform(X_test_b)

In [60]:
# train_ds = tf.data.Dataset.from_tensor_slices((a, y_train))
# test_ds = tf.data.Dataset.from_tensor_slices((b, y_test))
# train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
# test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()

In [82]:
alexnet = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(img_size,img_size,1)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(num_classes, activation='softmax')
])

In [85]:
alexnet.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
alexnet.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_59 (Conv2D)           (None, 30, 30, 96)        11712     
_________________________________________________________________
batch_normalization_59 (Batc (None, 30, 30, 96)        384       
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 14, 14, 96)        0         
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 14, 14, 256)       614656    
_________________________________________________________________
batch_normalization_60 (Batc (None, 14, 14, 256)       1024      
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 6, 6, 256)         0         
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 6, 6, 384)       

In [86]:
alexnet.fit(X_train_b.reshape(-1,img_size,img_size,1), y_train_b, batch_size=64, steps_per_epoch=30, epochs=30)

Epoch 1/30
30/30 [==============================] - 26s 863ms/step - loss: 3.4009 - accuracy: 0.3833
Epoch 2/30
30/30 [==============================] - 24s 812ms/step - loss: 1.0707 - accuracy: 0.4427
Epoch 3/30
30/30 [==============================] - 25s 822ms/step - loss: 1.0479 - accuracy: 0.4608
Epoch 4/30
30/30 [==============================] - 25s 818ms/step - loss: 1.0500 - accuracy: 0.4608
Epoch 5/30
30/30 [==============================] - 27s 899ms/step - loss: 1.0334 - accuracy: 0.4573
Epoch 6/30
30/30 [==============================] - 26s 850ms/step - loss: 1.0037 - accuracy: 0.4896
Epoch 7/30
30/30 [==============================] - 27s 905ms/step - loss: 1.0195 - accuracy: 0.4544
Epoch 8/30
30/30 [==============================] - 27s 893ms/step - loss: 1.0167 - accuracy: 0.4779
Epoch 9/30
30/30 [==============================] - 27s 887ms/step - loss: 1.0227 - accuracy: 0.4694
Epoch 10/30
30/30 [==============================] - 28s 917ms/step - loss: 1.0247 - accura

In [87]:
predictions = alexnet.predict(X_test.reshape(-1,img_size,img_size,1))
score = tf.nn.softmax(predictions[0])
print(score)

tf.Tensor([0.34711286 0.29945523 0.35343194], shape=(3,), dtype=float32)


In [99]:
alexnet.evaluate(X_test.reshape(-1,img_size,img_size,1), y_test_b)

19/19 [==============================] - 1s 66ms/step - loss: 1.2444 - accuracy: 0.4582


[1.2443822622299194, 0.45819398760795593]

In [88]:
vgg_like = keras.models.Sequential([
    keras.layers.Conv2D(16, 3, activation='relu', padding='same', input_shape=(img_size,img_size,1)),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.SeparableConv2D(32, 3, activation='relu', padding='same'),
    keras.layers.SeparableConv2D(32, 3, activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(),
    keras.layers.SeparableConv2D(64, 3, activation='relu', padding='same'),
    keras.layers.SeparableConv2D(64, 3, activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(),
    keras.layers.SeparableConv2D(128, 3, activation='relu', padding='same'),
    keras.layers.SeparableConv2D(128, 3, activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(),
    keras.layers.Dropout(0.2),
    keras.layers.SeparableConv2D(256, 3, activation='relu', padding='same'),
    keras.layers.SeparableConv2D(256, 3, activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.7),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(num_classes, activation='softmax')
])

In [89]:
vgg_like.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [90]:
vgg_like.fit(X_train_b.reshape(-1,img_size,img_size,1), y_train_b, batch_size=64, steps_per_epoch=30, epochs=30)

Epoch 1/30
30/30 [==============================] - 24s 803ms/step - loss: 1.6286 - accuracy: 0.3703
Epoch 2/30
30/30 [==============================] - 23s 753ms/step - loss: 1.4673 - accuracy: 0.3719
Epoch 3/30
30/30 [==============================] - 23s 758ms/step - loss: 1.3256 - accuracy: 0.3948
Epoch 4/30
30/30 [==============================] - 23s 767ms/step - loss: 1.3078 - accuracy: 0.3921
Epoch 5/30
30/30 [==============================] - 23s 759ms/step - loss: 1.2511 - accuracy: 0.3969
Epoch 6/30
30/30 [==============================] - 23s 751ms/step - loss: 1.2221 - accuracy: 0.4156
Epoch 7/30
30/30 [==============================] - 23s 751ms/step - loss: 1.1834 - accuracy: 0.4070
Epoch 8/30
30/30 [==============================] - 25s 840ms/step - loss: 1.1706 - accuracy: 0.4065
Epoch 9/30
30/30 [==============================] - 24s 790ms/step - loss: 1.1288 - accuracy: 0.4299
Epoch 10/30
30/30 [==============================] - 24s 811ms/step - loss: 1.1235 - accura

In [98]:
vgg_like.evaluate(X_test_b.reshape(-1,img_size,img_size,1), y_test_b)

19/19 [==============================] - 2s 94ms/step - loss: 1.0740 - accuracy: 0.4214


[1.0739824771881104, 0.4214046895503998]